In [1]:
import os
BASE_DIR = '../'
PATH_TO_TRAIN = os.path.join(BASE_DIR, 'basf/productos.csv')

In [ ]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv(PATH_TO_TRAIN, sep=';')

# Filtrar columnas relevantes
df = df[['nme_product', 'nme_target', 'nme_crop']]

# Eliminar duplicados y datos faltantes
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

# Verificar el conjunto de datos procesado
print(df.head())

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Cargar el modelo de lenguaje preentrenado
model = SentenceTransformer('all-MiniLM-L6-v2')

# Combinar columnas de plaga y cultivo para crear el contexto completo
df['context'] = df['nme_target'] + ' ' + df['nme_crop']

# Generar embeddings para cada contexto (combinación de plaga y cultivo)
context_embeddings = model.encode(df['context'], convert_to_tensor=True)

# Función para recomendar productos según cultivo y plaga
def recomendar_producto(cultivo, plaga, top_n=5):
    # Crear la consulta a partir del cultivo y la plaga
    query = plaga + ' ' + cultivo
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Calcular la similitud entre la consulta y los contextos
    cos_sim = util.pytorch_cos_sim(query_embedding, context_embeddings)

    # Obtener los índices de las mejores coincidencias
    top_results = cos_sim.argsort(descending=True).flatten()[:top_n]

    # Devolver los productos recomendados
    return df.iloc[top_results]['nme_product'].values

In [ ]:
productos_recomendados = recomendar_producto('Maíz', 'Isoca bolillera')
print(productos_recomendados)

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Cargar el modelo de lenguaje preentrenado
model = SentenceTransformer('all-MiniLM-L6-v2')

# Combinar columnas de plaga y cultivo para crear el contexto completo
df['context'] = df['nme_target'] + ' ' + df['nme_crop']
df.reset_index(drop=True, inplace=True)

# Generar embeddings para cada contexto (combinación de plaga y cultivo)
context_embeddings = model.encode(df['context'], convert_to_tensor=True)

# Función para recomendar productos según cultivo y plaga
def recomendar_producto(cultivo, plaga, top_n=5):
    # Crear la consulta a partir del cultivo y la plaga
    query = plaga + ' ' + cultivo
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Calcular la similitud entre la consulta y los contextos
    cos_sim = util.pytorch_cos_sim(query_embedding, context_embeddings)

    # Obtener los índices de las mejores coincidencias
    top_results = cos_sim.argsort(descending=True).flatten()

    # Manejar si hay menos de top_n resultados
    if len(top_results) == 0:
        return "No se encontraron productos coincidentes."
    else:
        # Limitar a top_n o a la cantidad de resultados disponibles
        top_results = top_results[:min(top_n, len(top_results))]

        # Devolver los productos recomendados
        return df.iloc[top_results]['nme_product'].values



In [ ]:
productos_recomendados = recomendar_producto('Maíz', 'Isoca bolillera')
print(productos_recomendados)